<a href="https://colab.research.google.com/github/mariajosemv/Exploring-the-Bitcoin-Cryptocurrency-Market/blob/main/getting_data_from_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Settings for Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks/DataCamp-Projects/exploring-cryptocurrencies'
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/DataCamp-Projects/exploring-cryptocurrencies
getting_data_from_api.ipynb


# Getting the data

The information about the current cryptocurrency market capitalization will be load from the [coinmarketcap API](https://coinmarketcap.com/). 

> **Important**: *If you want to replicate this code and get the current market cap, you must get another valid API-KEY on [CoinMarketCap](https://coinmarketcap.com/) and replace it in the code below.*




In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start': 1,
  'limit': 1000,
  'sort':'market_cap'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '09a08d1e-4090-485b-a0de-89b9c9584340',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(f"data loaded!")
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

data loaded!


Now that the data is downloaded, we briefly explore the dataset to see what we got. 

In [ ]:
import pandas as pd
import numpy as np 

df = pd.DataFrame(data['data'])

In [ ]:
df.head(3)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,last_updated,quote
0,1,Bitcoin,BTC,bitcoin,9747,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.859682e+07,1.859682e+07,None,1,2021-01-11T01:53:02.000Z,"{'USD': {'price': 36879.86458253049, 'volume_2..."
1,1027,Ethereum,ETH,ethereum,5947,2015-08-07T00:00:00.000Z,"[mineable, pow, smart-contracts]",NaN,1.142004e+08,1.142004e+08,None,2,2021-01-11T01:53:02.000Z,"{'USD': {'price': 1196.9983573740908, 'volume_..."
2,825,Tether,USDT,tether,9889,2015-02-25T00:00:00.000Z,"[store-of-value, payments, stablecoin, stablec...",NaN,2.427181e+10,2.478163e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,2021-01-11T01:53:07.000Z,"{'USD': {'price': 0.9997187185141, 'volume_24h..."


The dataset looks fine except for the columns `'tags'` and `'quote'`. The first one it will be not used so we can drop it, but the second one, i.e. quote column, it contains the vital information that is required for this analysis. Therefore, in this case we check the recent loaded JSON to identify the format that the `'quote'` feature has.

In [ ]:
data['data'][0]

{'circulating_supply': 18596825,
 'cmc_rank': 1,
 'date_added': '2013-04-28T00:00:00.000Z',
 'id': 1,
 'last_updated': '2021-01-11T01:53:02.000Z',
 'max_supply': 21000000,
 'name': 'Bitcoin',
 'num_market_pairs': 9747,
 'platform': None,
 'quote': {'USD': {'last_updated': '2021-01-11T01:53:02.000Z',
   'market_cap': 685848387665.0176,
   'percent_change_1h': -4.17062327,
   'percent_change_24h': -9.46465197,
   'percent_change_7d': 12.83371557,
   'price': 36879.86458253049,
   'volume_24h': 79448280993.27394}},
 'slug': 'bitcoin',
 'symbol': 'BTC',
 'tags': ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channels'],
 'total_supply': 18596825}

Here, we see that `'quote'` contains the keys and values required for this project, so then, the next step is to extract this information and save it in another dataframe for later concatenate with the main one. 

In [ ]:
df_mc = pd.DataFrame()


for i in np.arange(len(data['data'])):
  index_ = data['data'][i]['quote']['USD']
  df_i = pd.DataFrame.from_dict([index_])
  df_mc = pd.concat([df_mc, df_i], sort=False)
  df_mc.reset_index(drop=True, inplace=True) 


In [ ]:
df_mc.head()

,price,volume_24h,percent_change_1h,percent_change_24h,percent_change_7d,market_cap,last_updated
0,36879.864583,7.944828e+10,-4.170623,-9.464652,12.833716,6.858484e+11,2021-01-11T01:53:02.000Z
1,1196.998357,3.932000e+10,-5.533442,-7.430299,22.447485,1.366977e+11,2021-01-11T01:53:02.000Z
2,0.999719,1.341662e+11,-0.045812,-0.221235,-0.023216,2.426499e+10,2021-01-11T01:53:07.000Z
3,0.302537,9.538569e+09,-4.774535,-9.312490,32.802597,1.373638e+10,2021-01-11T01:54:02.000Z
4,164.493605,1.231376e+10,-4.048837,-7.708016,3.453726,1.089698e+10,2021-01-11T01:54:02.000Z


The dataframe looks very well, so we can finally concat `df_mc` with the principal dataframe `df`, and drop the undesired columns

In [ ]:
df = pd.concat([df, df_mc],axis=1)

In [ ]:
df.columns

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply', 'platform',
       'cmc_rank', 'last_updated', 'quote', 'price', 'volume_24h',
       'percent_change_1h', 'percent_change_24h', 'percent_change_7d',
       'market_cap', 'last_updated'],
      dtype='object')

In [ ]:
df.drop(['quote', 'last_updated', 'tags'], axis=1, inplace=True)

In [ ]:
df.to_csv('/datasets/coinmarketcap_10_01_2021.csv')